Web Scrapping to build the ideal Machine Learning/ Deep Learning Course

# Installing Libraries

In [1]:
#!pip install bs4
#!pip install numpy
#!pip install pandas

#!pip install ast
#!pip install matplotlib
#!pip install nltk
#!pip install wordcloud

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np
import nltk
import nltk
import os
import csv
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
#from hcluster import pdist, linkage, dendrogram
from nltk.tokenize import RegexpTokenizer
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import pairwise_distances

import scipy.cluster.hierarchy as sch



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Online Course Recommender

## Scraping CourseReport.com to obtain course information and ratings in data science bootcamps, and online education platforms.



Sraping course report for course postings and rankings to be linked to the relevant skills required. Starts by obtaining all the links for each page. Next iterates through each page to obtain Datascience Course program/website, rating and # of reviews, and links to further investigate courses later.

In [3]:

#create an empty DF 
course_table = pd.DataFrame()
URL = 'https://www.coursereport.com/tracks/data-science-bootcamp'
page = requests.get(URL)
        #specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, 'html.parser')
#creating a blank list for links
links = []
#web scrapping for links of each page containing relevant data
for divTag in soup.find_all('nav',{'class': 'pagination'}):
    for aTag in divTag.find_all('a'):
        links.append(aTag.get('href'))
        time.sleep(0.5)
links = links[:-1]
#noticed the link for page 2 was grabbed twice instead of page 1 so replaced with the link for page 1
links[0] = '/tracks/data-science-bootcamp'
#resetting the base so the proceeding loop can modify to each specific page
URL = 'https://www.coursereport.com'

for link in links:
    URL = 'https://www.coursereport.com'
    URL += link 
    page = requests.get(URL)
        #specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
    soup = BeautifulSoup(page.text, 'html.parser')
#looping though each course posting and obtaining relevant info
    for courses in soup.find_all("li", class_ = "school-li"): 
        #sleep(1)
        try:
            course_title = courses.find('div', class_ = 'school-header').text.strip()
      #print(job_title)
        except:
            course_title = None

        try:
            rating = courses.find('p', class_ = 'rating-number').text.strip()
        except:
            rating = None

        try:
            link = courses.find('div', attrs={'class': 'school-header'}).find('a', href=True)['href']
        except:
            link = None   
        
        link+='#/courses'
        time.sleep(0.5)
        #building the dataframe row by row 
        course_table = course_table.append({
            'course_title': course_title, 
            'avg_rating': rating,
            'link': link}, ignore_index = True)

### Scraping Links to get summaries

In [4]:
# course_table

## Filtering ratings

Below, the data was cleaned to get the rating and # of reviews in there own respective columns. Many postings that were featured on the website included that in the course platform  title, so this was removed. As avg_rating is a string the columns that were made for reviews and ratings were converted to numeric.

In [5]:
#course_table.head()

#course_table = course_table.replace(np.nan, '', regex=True)
course_table = course_table[course_table['avg_rating'].notna()]
course_table['course_title'] = course_table['course_title'].str.replace('Featured','')
course_table['avg_rating'] = course_table['avg_rating'].str.replace('Avg Rating:','')
course_table['avg_rating'] = course_table['avg_rating'].str.replace(' reviews','')
course_table['avg_rating'] = course_table['avg_rating'].str.replace(')','')
#course_table['avg_rating'] = course_table['avg_rating'].str.replace('(','')


course_table[['avg_rating','reviews']] = course_table['avg_rating'].str.split('(',expand=True,)

In [6]:
#course_table.tail()


#converting string to numeric
course_table[['avg_rating','reviews']] = course_table[['avg_rating','reviews']].apply(pd.to_numeric)


#course_table[course_table['reviews'] > 100].sort_values(by='avg_rating', ascending=False).head(10)



course_sifter = course_table[course_table['reviews'] > 150].sort_values(by='avg_rating', ascending=False)
#course_sifter = course_sifter.iloc[0:20,:]
course_sifter = course_sifter.reset_index()
#course_sifter

for i in range(len(course_sifter)):
    link=course_sifter.iloc[i,3]
    prefix='https://www.coursereport.com'
    course_sifter.iloc[i,3]=prefix+link

#course_sifter.iloc[i,3]

program_table = pd.DataFrame()
for i in range(len(course_sifter)):
    detail = requests.get(course_sifter.iloc[i,3]) 
    soup = BeautifulSoup(detail.text, "lxml")
    for courses in soup.find_all("ul", class_ = "course-card"): 
        try:
            program_title = courses.find('h3').text.strip()
      #print(job_title)
        except:
            program_title = None

        try:
            skills = courses.find('div', class_ = 'details').text.strip()
        except:
            skills = None
        try:
            course_link = courses.find('span', attrs={'class': 'btn btn-default btn-apply small'}).find('a', href=True)['href']
        except:
            course_link = None
        
        time.sleep(0.5)
        #building the dataframe row by row 
        program_table = program_table.append({
            'course_title': course_sifter.iloc[i,2],
            'program_title': program_title, 
            'skills': skills,
            'avg_rating': course_sifter.iloc[i,1],
            'reviews': course_sifter.iloc[i,4],
            'course_link': course_link}, ignore_index = True)

program_table.head()

,avg_rating,course_link,course_title,program_title,reviews,skills
0,4.99,,Le Wagon,Data Science Course - Full-time,1719.0,"Data Science, Git, Data Visualization, Data An..."
1,4.99,http://www.lewagon.com/apply,Le Wagon,Web Development Course - Full-time,1719.0,"MySQL, HTML, Git, JavaScript, Sinatra, jQuery,..."
2,4.99,,Le Wagon,Web Development Course - Part-time,1719.0,"MySQL, HTML, Git, JavaScript, Sinatra, jQuery,..."
3,4.99,None,Le Wagon,None,1719.0,None
4,4.99,None,Le Wagon,None,1719.0,None


In [7]:
# #copier=program_table
# copier=program_table
# program_table=copier


In [8]:
#program_table

In [9]:
# print(program_table.iloc[0,4])

In [10]:
# #program_table.to_csv(r'C:\Users\Tyler\Documents\Python notes\Project\wtf.csv', index = False)

In [11]:

# # program_table.columns = program_table.columns.str.strip()
# # program_table = program_table[program_table.skills != None]
# # program_table = program_table[program_table.skills != ""]
# # program_table.dropna()
# program_table.drop_duplicates(keep=False,inplace=True) 
# program_table = program_table.reset_index(drop=True)

In [12]:
# program_table.head()

## Obtaining list of acceptable courses from highly rated schools

The following list named skillset was created in order to search the schools for keywords that are relavent towards AI,ML and data science...

In [13]:
skillset = ['deep learning', 'machine learning', 'ai' ,'artificial intelligence', 'ml','data science', 'cost analysis','prediction','code','coding','data architecture','programming','machine learning','ai','statistics','mathematic','communication','critical thinking','visualization','master','professional','bachelor','business decisions','operationalizing','infrastructure','prototype','workflows','modeling','statistical','cloud','jupyter','rstudio','pycharm','atom','matlab','visual studio','spyder','vim','notepad','sublime','google colab','google','google cloud','notebook','ai platform','datalab','binder','ibm watson','ibm watson studio','ocean','python','r','sql','c','c++','java','javascript','typescript','bash','ggplot','matplotlib','plotly','d3','shiny','seaborn','geoplotlib','leaflet','cpu','gpu','tpu','linear','logistic','regression','forest','gradient','bayesian','neural networks','neural','dense','convolutional','transformer','ml','augmentation','automated','feature engineering','selection','architecture','pipelines','image','video','pil','cv2','skimage','u-net','mask r-cnn','image segmentation','object detection  ','yolov3','retinanet','image classification','vgg','inception','resnet','gan','vae','computer vision','generative networks','word embeddings','vectors','glove','fasttext','word2ve','encoder','decorder seq2seq','vanilla','contextualized embeddings','elmo','cove','gpt-2','bert','xlnet','scikit-learn','tensorflow','keras','randomforest','xgboost','pyTorch','caret','lightgbm','spark mlib','fast.ai','amazon','azure','salesforce','alibaba','aws']
#list of skills]

program_table['descript']='skills'
for skill in skillset:
    program_table[skill] = np.zeros(len(program_table))
#reset_index
program_table = program_table.reset_index()
#program_table.head(5)

program_table['cleaned_course']='blank'
program_table['cleaned_skills']='blank'
    

Now I will search if the list of key words defined as "skillset' appear in either the course title or description....

In [14]:
tokenized=[]
for i in range(len(program_table)):
    try:
        text = str(program_table.iloc[i,6])
        
        string = re.sub(r'\,', ' ', text) # remove ","
        string = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', string) # spacing betweeen capitals "MachineLearning"
        string = string.lower() # converting everything to lower case
        #string = re.sub(r'\n',' ',string) 
        string = re.sub(r'\,', ' ', string) # remove ","
        string = re.sub('/', ' ', string) # remove "/"
       
        string = re.sub(r'\(', ' ', string) # remove "("
        string = re.sub(r'\)', ' ', string) # remove ")"
        string = re.sub(' +',' ',string) # remove more than one space  
        
        
        text2 = str(program_table.iloc[i,4])
        string2 = re.sub(r'\,', ' ', text2) # remove ","
        string2 = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', string2)
        string2 = string2.lower()
        #string2 = re.sub(r'\n',' ',string2) 
        string2 = re.sub(r'\,', ' ', string2) # remove ","
        string2 = re.sub('/', ' ', string2) # remove "/"
        string2 = re.sub(r'\(', ' ', string2) # remove "("
        string2 = re.sub(r'\)', ' ', string2) # remove ")"
        string2 = re.sub(' +',' ',string2) # remove more than one space  
        
                
    except:
        text = None
        text = "" 
        text2 = None
        text2 = ""
    print(string2)
    program_table['cleaned_course'][i] = string2
    program_table['cleaned_skills'][i] = string
    #program_table.iloc[i,4] = string2
    
    for sk in skillset :
        
        if any(x in sk for x in ['+']):
            skk = re.escape(sk)
        else:
            skk = sk
        result = re.search(r'(?:^|(?<=\s))' + skk + r'(?=\s|$)',string)
        result2 = re.search(r'(?:^|(?<=\s))' + skk + r'(?=\s|$)',string2)
        
        if result:

            program_table_text = 1
            
        else:
            program_table_text = 0
        
        if result2:    
            program_table_text2 = 1
            
        else:
            program_table_text2 = 0

        program_table[sk][i] = program_table_text+program_table_text2

data science course - full-time
web development course - full-time
web development course - part-time
none
none
none
none
none
none

C:\Users\Tyler\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Tyler\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Tyler\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



none
none
none
none
none
none
data science masters program - remote
data science masters program with deep learning
none
none
none
none
none
none
none
none
none
none
none
none
12-weeks in-person data science bootcamp
big data with amazon cloud hadoop spark and docker
data science with python: data analysis and visualization weekend course 
data science with python: machine learning weekend course 
data science with r: data analysis and visualization weekend course 
data science with r: machine learning weekend course 
deep learning with tensorflow weekends and in-person only 
full-time online data science bootcamp
introductory python evenings 
part-time online data science bootcamp
none
none
none
none
none
none
none
none
none
none
none
none
in-person c# and .net framework boot camp full-time 
in-person c# and .net framework boot camp part-time 
in-person data science boot camp full-time 
in-person data science boot camp part-time 
in-person python boot camp full-time 
in-person python

In [15]:
# program_table.iloc[1,1]

In [16]:
# program_table.head()

## Collecting dataframes for recommender

After trying to filter out some of the results I still have too many options and it can be seen that some of the options may only contain data science, or ML/AI. To filter more I will make them require definitely require some aspect of Machine Learning first... and I can further reduce later.

In [17]:
# recommended_courses = program_table

In [18]:
#recommended_courses.to_csv(r'C:\Users\Tyler\Documents\Python notes\Project\recommended_courses.csv', index = False)

In [19]:
job_list = pd.read_csv(r'./ZipRecruiter.csv')
recommended_courses = pd.read_csv(r'./recommended_courses.csv')

In [20]:
skillset = ['deep learning', 'machine learning', 'ai' ,'artificial intelligence', 'ml','data science', 'cost analysis','prediction','code','coding','data architecture','programming','machine learning','ai','statistics','mathematic','communication','critical thinking','visualization','master','professional','bachelor','business decisions','operationalizing','infrastructure','prototype','workflows','modeling','statistical','cloud','jupyter','rstudio','pycharm','atom','matlab','visual studio','spyder','vim','notepad','sublime','google colab','google','google cloud','notebook','ai platform','datalab','binder','ibm watson','ibm watson studio','ocean','python','r','sql','c','c++','java','javascript','typescript','bash','ggplot','matplotlib','plotly','d3','shiny','seaborn','geoplotlib','leaflet','cpu','gpu','tpu','linear','logistic','regression','forest','gradient','bayesian','neural networks','neural','dense','convolutional','transformer','ml','augmentation','automated','feature engineering','selection','architecture','pipelines','image','video','pil','cv2','skimage','u-net','mask r-cnn','image segmentation','object detection  ','yolov3','retinanet','image classification','vgg','inception','resnet','gan','vae','computer vision','generative networks','word embeddings','vectors','glove','fasttext','word2ve','encoder','decorder seq2seq','vanilla','contextualized embeddings','elmo','cove','gpt-2','bert','xlnet','scikit-learn','tensorflow','keras','randomforest','xgboost','pyTorch','caret','lightgbm','spark mlib','fast.ai','amazon','azure','salesforce','alibaba','aws']
#list of skills]

In [21]:
jobs=job_list.iloc[:,1:6]
columns_titles = ["company","job_title",'location','salary','link']
jobs=jobs.reindex(columns=columns_titles)
jobs= jobs[0:7]
#job_list
jobs

,company,job_title,location,salary,link
0,rockwell automation,Software Data Architect,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/eclk/fAQRWYLiCODH...
1,PagerDuty,Senior Data Engineer - Analytics,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/eclk/PrjVFPG3P6gb...
2,Goodfood Market Corp.,BI Engineer,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/c/Goodfood-Market...
3,Interac,Senior Data Engineer,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/eclk/gptSw66Ty4w_...
4,Project X,Consultant- Big Data Developer (4 Positions),"Toronto, ON",Full-Time,https://www.ziprecruiter.com/c/Project-X/Job/C...
5,Rocket Homes,Big Data Engineer,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/eclk/3yN3BWOj-n7Z...
6,Serigor Inc.,Informatica MDM Architect,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/c/Serigor-Inc./Jo...


In [22]:
recommended_courses['score']=0
skillset

['deep learning',
 'machine learning',
 'ai',
 'artificial intelligence',
 'ml',
 'data science',
 'cost analysis',
 'prediction',
 'code',
 'coding',
 'data architecture',
 'programming',
 'machine learning',
 'ai',
 'statistics',
 'mathematic',
 'communication',
 'critical thinking',
 'visualization',
 'master',
 'professional',
 'bachelor',
 'business decisions',
 'operationalizing',
 'infrastructure',
 'prototype',
 'workflows',
 'modeling',
 'statistical',
 'cloud',
 'jupyter',
 'rstudio',
 'pycharm',
 'atom',
 'matlab',
 'visual studio',
 'spyder',
 'vim',
 'notepad',
 'sublime',
 'google colab',
 'google',
 'google cloud',
 'notebook',
 'ai platform',
 'datalab',
 'binder',
 'ibm watson',
 'ibm watson studio',
 'ocean',
 'python',
 'r',
 'sql',
 'c',
 'c++',
 'java',
 'javascript',
 'typescript',
 'bash',
 'ggplot',
 'matplotlib',
 'plotly',
 'd3',
 'shiny',
 'seaborn',
 'geoplotlib',
 'leaflet',
 'cpu',
 'gpu',
 'tpu',
 'linear',
 'logistic',
 'regression',
 'forest',
 'gradien

## Recommender Function

In [23]:
def recommender_system (jb, rc, t, skill):
    rc['score']=0
    
    for i in range(len(rc)):
        for sk in skill:
            #print (sk)
            if rc[sk][i]>0 and jb[sk][t]>0:
                rc['score'][i]+=1
                
            else:        
                pass
    columns_titles = ["course_title","program_title",'avg_rating', 'course_link']
    rc = rc.sort_values(by='score', ascending=False)
    rc=rc.reindex(columns=columns_titles)
    recommendation = rc.iloc[0:3,0:5]
    return(recommendation)
    

In [24]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## Get started on your quest for your dream job !

In [25]:

jobs

,company,job_title,location,salary,link
0,rockwell automation,Software Data Architect,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/eclk/fAQRWYLiCODHDVhTHh7rb3onxGwlezFluFwVDL8YavB0CNp3AbuQfM0sDZbig-9f-gum4QBlMcwzOnjKXsvDSNZ6PuK3x_6sMLu-Yd1n3yjlTC1eQK-VMffepbYYkLKSbjQDbe5aKZKbQ2sUFGNgIp9ogk14YLbG3d1Qok5tWgsGDLWH3AlxcurcCXOKD6fpYOmDChJCcaPkyk_xsRVDCrzpMvtZG6i61rgxUcrA4szDs49gLYXwQQ1NwrppaGm-QwH8xVbsf3y5X4DkOLvoh-3r55vd5hNG9Q2lEQPajniqNFcxex5s7C5cor3669MmG4KiMXPPDNJdQXTI6jaK9Ejp0cHxpp6hihZd3PPWYmu2ySIzAOQ2YB_jz4ca0GzOMTQDyjaDJR9psJQ-lkAtQBefMwtc_YPdICFGKDGMxYRmlpHfuEDzW1Oi0ssrvENVJkxABjz1ps6goB8h22erF0bJy2UDOavIU7b9OHtlpVQ=.7ea07f6c767dc9c9b3a4367cd39e8266
1,PagerDuty,Senior Data Engineer - Analytics,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/eclk/PrjVFPG3P6gbwN0p_i0XWDKFgvBTNJrNRGGKDCMhHL7mRdXRM2b5Fmi_Haclq2VKQIub7up0I3OtddG3cK9rw-1YrwqchEzOYeon4r4uv4Qth2rf_pRqM4aiba5Qaz-2y-FPjaOMRNdxpK9Xwkt9SPbHGPc12RmDQHtgP7wTMSWTGFRZ3-nTfvGF9M3SaroLJ8WZkQbFBsigjbfGxTVJlYiFyZDB2fuCniKFo_L_gvoiq9aYP8ApQKz2P0-urNfDMJmzmT4WBc0oAtBe8_gRz8CDYiLhn7tQFRQZRdeYgiVv0cMxbdi1-4PnJVdNMt_wqMQOeLgwOElk5my26BQFMxODZWIiuoBNkDml6ssPRdmwS5vxuCRwHkbWdE2npezo9BQ_-0SOEIB8YP4U8Qx7rb4vn1l_COOWr6GoBwEiLhNSE9OVth16dFeqJXlG_Mq0VwDuYQBGvKvz_uNm4pLKTFFGe3TvtoB9wRxkHWqfgG4=.d27744abe62985b25a8a5bd9c7ca89d4
2,Goodfood Market Corp.,BI Engineer,"Toronto, ON",Full-Time,"https://www.ziprecruiter.com/c/Goodfood-Market-Corp./Job/BI-Engineer/-in-Toronto,ON?jid=DQ0ea4828fc2fb1b087020654f11f47678&job_id=331a1f2c39c037a7e69b2aca0c24f539"
3,Interac,Senior Data Engineer,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/eclk/gptSw66Ty4w_7ZFRLIsXd4hQdYNxZ7KUDleAudN3vWWS1P70Em6ybBtL18O2rnGybTjbyVT6vDcor00ysOMuEAJXF89hauACXEHluf-WUw_NwCYx8F8hRty4jWB6VhZa4sdtgCAbclp-0hBs8Wsy0vxU7fZLkaF6_hW2HG9V7k8puGW6tWS0P8wBf_OtiNiY_VplUhHH1UPUObVO9a6rd9lT2pphogQLw92gM2HdkHwo_cuy_i5WULtgGIsH7uI4aIQ5wPwk6oWFYGXXwX-ebrO94kIZ6X0XMA3U-w50GuFGUJTZlsf-hNxwPh0zp5SpKTvGHEKFS2GodGpw97e3NWmxBLn7e1HlNS7ecSIsZfza_GufJpE5P6xg6Ak99_KA0zl112DAFEs4Z9JzFjEaesjmwiPpCW4DpWrYatMg1ZHRdehPsY3YECkYwXXTS5WZIkmDwr8KXJuJagnsBvXDUB6A5Gh_-1WU3fg-sirmpJA=.46efbfc84a9cf59794f4abce5fe8b7ab
4,Project X,Consultant- Big Data Developer (4 Positions),"Toronto, ON",Full-Time,"https://www.ziprecruiter.com/c/Project-X/Job/Consultant-Big-Data-Developer-(4-Positions)/-in-Toronto,ON?ojob=88a1b281c09d749f83a63c440961843b"
5,Rocket Homes,Big Data Engineer,"Toronto, ON",Full-Time,https://www.ziprecruiter.com/eclk/3yN3BWOj-n7ZC7ZDF6l4i12xAdbDU8bKHuIi2u8hIIMPgXuNJQG4uwt9jW7Wl9KqxykqMKei5y7UKVcsZeNlCt-EzDThZFgrBJAI-g4CsxJNm_SNQC-zTWrzAu1ANxNhe3EvLmBECoPHWpXiV65tGDX2aqfJ09D839qiKtwsW9ifPSEPnoyUw6f1m2KGUxQl0cgMF64Dipgaqxjb_Ruu0PFJDmt2okhds-UttejfiCPItpd4pqnPb4rP2rzL1g5o1HtqXEbRMRHZVCtiWeD4BNsQ77iTD4K7zlcI9QxRjMH2LKK0aiKuXMIZkzTG63mcz7NpYcM6ZWQCYdlqlc07M6zAju8U7k0hvIb01r_0Yc6uJnLivpSueHS7IYeMsvOni2fdhz-U0-03iBDBQNhe8aqxtP5Jpk0523_Jc_NSbeaM53p_6glR1Uw7YPPADMT2SMMcs146gJLYiPnadVxCco_G_4Ygj0WhS6wVELvl18c=.11067628bd1dbbd45c341b10ee097138
6,Serigor Inc.,Informatica MDM Architect,"Toronto, ON",Full-Time,"https://www.ziprecruiter.com/c/Serigor-Inc./Job/Informatica-MDM-Architect/-in-Toronto,ON?ojob=10bdb112e57e79adedfc0d991c3a0c4b"


Welcome to Course Compass! Please look through the company names and jobs and enter the one that interest you below:

In [26]:
#Type line of job beside X and click and run !
x=4
bang= recommender_system (job_list, recommended_courses, x, skillset)
print('The top 3 courses for you to get that dream job at:', jobs.iloc[x,0], ' in ' ,jobs.iloc[x,2], ' as a ', jobs.iloc[x,1], ' at a ', jobs.iloc[x,3], ' rate:')
bang


C:\Users\Tyler\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


The top 3 courses for you to get that dream job at: Project X  in  Toronto, ON  as a  Consultant- Big Data Developer (4 Positions)  at a  Full-Time  rate:


,course_title,program_title,avg_rating,course_link
116,BrainStation,Introduction to Cloud Computing Training Online,4.67,https://brainstation.io/training/intro-cloud-computing?utm_source=CourseReport&utm_medium=referral&utm_campaign=cloudComputing_online
48,Flatiron School,Online Software Engineering Immersive,4.71,https://flatironschool.com/career-courses/coding-bootcamp/online/?utm_campaign=Sponsored_Content&utm_source=Course%20Report&utm_medium=FS%20Profile%20-%20OSE%20PDP
61,Udacity,Cloud Developer,4.71,https://www.udacity.com/course/cloud-developer-nanodegree--nd9990
